In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# E2E ML on GCP: MLOps stage 2 : experimentation: get started with Vertex Training for Pytorch

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_pytorch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_template.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This tutorial demonstrates how to use Vertex AI for E2E MLOps on Google Cloud in production. This tutorial covers stage 2 : experimentation: get started with Vertex Training for Pytorch.

### Dataset

The dataset used for this tutorial is the [CIFAR10 dataset](https://pytorch.org/vision/stable/datasets.html#cifar) from [Pytorch Datasets](https://pytorch.org/vision/stable/datasets.html). The version of the dataset you will use is built into TensorFlow. The trained model predicts which type of class an image is from ten classes: airplane, automobile, bird, cat, deer, dog, frog, horse, ship, or truck.

### Objective

In this tutorial, you learn how to use `Vertex AI Training` for training a Pytorch custom model.

This tutorial uses the following Google Cloud ML services:

* `Vertex AI Training`
* `Vertex AI Model` resource


The steps performed include:

- Single node training using a Python package.
- Report accuracy when hyperparameter tuning.
- Save the model artifacts to Cloud Storage using GCSFuse.
- Create a `Vertex AI Model` resource.

## Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Google Cloud Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

## Installations

Install *one time* the packages for executing the MLOps notebooks.

In [1]:
! pip3 install --upgrade google-cloud-aiplatform[tensorboard] $USER_FLAG
! pip3 install --upgrade google-cloud-pipeline-components $USER_FLAG
! pip3 install --upgrade cloudml-hypertune $USER_FLAG
! pip3 install --upgrade torchvision $USER_FLAG

     |████████████████████████████████| 489.7 MB 5.3 kB/s  eta 0:00:01     |██████████████▌                 | 221.8 MB 61.2 MB/s eta 0:00:05MB/s eta 0:00:03     |████████████████████            | 306.1 MB 26.0 MB/s eta 0:00:08██████████████▉           | 318.9 MB 26.0 MB/s eta 0:00:07�█▎ | 463.7 MB 47.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 56.7 MB/s eta 0:00:01
     |████████████████████████████████| 463 kB 70.1 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 4.6 MB/s  eta 0:00:01
     |████████████████████████████████| 4.5 MB 43.1 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 7.8 MB/s  eta 0:00:01
     |████████████████████████████████| 5.8 MB 46.2 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 6.7 MB/s  eta 0:00:01
     |████████████████████████████████| 2.1 MB 49.7 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.7 MB/s  eta 0:00:01
     |████████████████████████████████| 14.5 MB 23.6 MB/s eta 

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [2]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [1]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

Project ID: vertex-ai-dev


In [3]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update



To take a quick anonymous survey, run:
  $ gcloud survey



#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [4]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [5]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [6]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}

In [7]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [8]:
! gsutil mb -l $REGION $BUCKET_URI

Creating gs://vertex-ai-devaip-20220324060506/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [9]:
! gsutil ls -al $BUCKET_URI

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [28]:
import google.cloud.aiplatform as aiplatform

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [29]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### Set hardware accelerators

You can set hardware accelerators for training.

Set the variable `TRAIN_GPU/TRAIN_NGPU` to use a container image supporting a GPU and the number of GPUs allocated to the virtual machine (VM) instance. For example, to use a GPU container image with 4 Nvidia Telsa K80 GPUs allocated to each VM, you would specify:

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

Otherwise specify `(None, None)` to use a container image to run on a CPU.

Learn more [here](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators) hardware accelerator support for your region

In [30]:
import os
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

#### Set pre-built containers

Set the pre-built Docker container image for training.

- Set the variable `TF` to the TensorFlow version of the container image. For example, `2-1` would be version 2.1, and `1-15` would be version 1.15. The following list shows some of the pre-built images available:


For the latest list, see [Pre-built containers for training](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers).

In [13]:
if TRAIN_GPU:
    TRAIN_VERSION = "pytorch-gpu.1-9"
else:
    TRAIN_VERSION = "pytorch-xla.1-9"

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)

#### Set machine type

Next, set the machine type to use for training.

- Set the variable `TRAIN_COMPUTE` to configure  the compute resources for the VMs you will use for for training.
 - `machine type`
     - `n1-standard`: 3.75GB of memory per vCPU.
     - `n1-highmem`: 6.5GB of memory per vCPU
     - `n1-highcpu`: 0.9 GB of memory per vCPU
 - `vCPUs`: number of \[2, 4, 8, 16, 32, 64, 96 \]

*Note: The following is not supported for training:*

 - `standard`: 2 vCPUs
 - `highcpu`: 2, 4 and 8 vCPUs

*Note: You may also use n2 and e2 machine types for training and deployment, but they do not support GPUs*.

In [14]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

Train machine type n1-standard-4


## Introduction to Pytorch training

The Pytorch package supports both single node and distributed model training.

Once you have trained a Pytorch model, you will want to save it at a Cloud Storage location, so it can subsequently be uploaded to a `Vertex AI Model` resource.
The Pytorch package does not have support to save the model to a Cloud Storage location. Instead, you will do the following steps to save to a Cloud Storage location.

1. Save the in-memory model to the local filesystem (e.g., model.pth).
2. Use gsutil to copy the local copy to the specified Cloud Storage location.

*Note*: You can do hyperparameter tuning with a Pytorch model.

### Examine the training package

#### Package layout

Before you start the training, you will look at how a Python package is assembled for a custom training job. When unarchived, the package contains the following directory/file layout.

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

The files `setup.cfg` and `setup.py` are the instructions for installing the package into the operating environment of the Docker image.

The file `trainer/task.py` is the Python script for executing the custom training job. *Note*, when we referred to it in the worker pool specification, we replace the directory slash with a dot (`trainer.task`) and dropped the file suffix (`.py`).

#### Package Assembly

In the following cells, you will assemble the training package.

In [15]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

#Instructions for installing package into environment of the docker image
setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'cloudml-hypertune',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: CIFAR10 image classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### Create the task script for the Python training package

Next, you create the `task.py` script for driving the training package. Some noteable steps include:

- Command-line arguments:
    - `model-dir`: The location to save the trained model. When using Vertex AI custom training, the location will be specified in the environment variable: `AIP_MODEL_DIR`,
    - `batch_size`/`lr` : Hyperparameter tuning variables
    - `distribute`: single node or distributed training.
- Data preprocessing (`get_data()`):
    - Download the dataset and split into training and test.
- Model architecture (`getmodel()`):
    - Get or build the model architecture.
- Training (`train_model()`):
    - Trains the model
- Evaluation (`evaluate_model()`):
    - Evaluates the model.
    - If hyperparameter tuning, reports the metric for accuracy.
- Model artifact saving
    - Saves the model artifacts and evaluation metrics where the Cloud Storage location specified by `model-dir`.

In [16]:
%%writefile custom/trainer/task.py
import sys
import os
import argparse
import logging
import hypertune

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parallel import DistributedDataParallel
#import torch.backends.cudnn as cudnn
import torch.distributed as distributed
#import torch.optim
#import torch.multiprocessing as mp
import torch.utils.data as data
#import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

parser = argparse.ArgumentParser(description='PyTorch ImageNet Training')
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--batch_size', dest='batch_size',
                    type=int, default=16, help='Batch size')
parser.add_argument('--epochs', dest='epochs',
                    type=int, default=20, help='Number of epochs')
parser.add_argument('--lr', dest='lr',
                    type=int, default=20, help='Learning rate')
parser.add_argument('--distribute', default="single",
                    type=str, help='Distributed training strategy')
parser.add_argument('--checkpoints', default=False,
                    type=bool, help='Whether to save checkpoints')
args = parser.parse_args()

logging.getLogger().setLevel(logging.INFO)

def distributed_is_initialized():
    if args.distribute == "mirror":
        if distributed.is_available() and distributed.is_initialized():
            return True
    return False

def get_data():

    normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    transform = transforms.Compose(
        [transforms.ToTensor(), normalize,]
    )

    train_dataset = datasets.CIFAR10(root="./train", transform=transform, train=True, download=True)
    logging.info(train_dataset)
    if distributed_is_initialized():
        sampler = data.DistributedSampler(train_dataset)
    else:
        sampler = None
    train_loader = data.DataLoader(
        train_dataset,
        batch_size=args.batch_size,
        shuffle=(sampler is None),
        sampler=sampler,
    )

    test_dataset = datasets.CIFAR10(root="./test", transform=transform, train=False, download=True)
    logging.info(test_dataset)
    sampler = None
    test_loader = data.DataLoader(
        test_dataset,
        batch_size=args.batch_size,
        shuffle=False,
        sampler=sampler,
    )

    return train_loader, test_loader

def get_model():
    class Cifar10Model(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(3, 16, 3)
            self.pool = nn.MaxPool2d(2, 2)
            self.conv2 = nn.Conv2d(16, 16, 5)
            self.fc1 = nn.Linear(16 * 5 * 5, 120)
            self.fc2 = nn.Linear(120, 84)
            self.fc3 = nn.Linear(84, 10)

        def forward(self, x):
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            x = torch.flatten(x, 1) # flatten all dimensions except batch
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            return x

    logging.info("Get model architecture")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    gpu_id = "0" if torch.cuda.is_available() else None
    logging.info(f"Device: {device}")

    model = Cifar10Model()
    model.to(device)

    if distributed_is_initialized():
        model = DistributedDataParallel(model)

    loss = nn.CrossEntropyLoss().cuda(gpu_id)
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    return model, loss, optimizer, device

def train_model(model, loss_func, optimizer, train_loader, test_loader, is_chief, device):

    class Average(object):
        def __init__(self):
            self.sum = 0
            self.count = 0

        def __str__(self):
            return '{:.6f}'.format(self.average)

        @property
        def average(self):
            return self.sum / self.count

        def update(self, value, number):
            self.sum += value * number
            self.count += number

    class Accuracy(object):
        def __init__(self):
            self.correct = 0
            self.count = 0

        def __str__(self):
            return '{:.2f}%'.format(self.accuracy * 100)

        @property
        def accuracy(self):
            return self.correct / self.count

        @torch.no_grad()
        def update(self, output, target):
            pred = output.argmax(dim=1)
            correct = pred.eq(target).sum().item()

            self.correct += correct
            self.count += output.size(0)

    def train():

        model.train()

        train_loss = Average()
        train_acc = Accuracy()

        for data, target in train_loader:
            data = data.to(device)
            target = target.to(device)

            output = model(data)
            loss = loss_func(output, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss.update(loss.item(), data.size(0))
            train_acc.update(output, target)

            return train_loss, train_acc

    @torch.no_grad()
    def evaluate(epoch):
        model.eval()

        test_loss = Average()
        test_acc = Accuracy()

        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)

        output = model(data)
        loss = loss_func(output, target)

        test_loss.update(loss.item(), data.size(0))
        test_acc.update(output, target)

        # report metric for hyperparameter tuning
        hpt = hypertune.HyperTune()
        hpt.report_hyperparameter_tuning_metric(
            hyperparameter_metric_tag='accuracy',
            metric_value=test_acc.accuracy,
            global_step=epoch
        )

        return test_loss, test_acc

    for epoch in range(1, args.epochs + 1):

        logging.info('Epoch: {}, Training ...'.format(epoch))
        train_loss, train_acc = train()

        if is_chief:
            test_loss, test_acc = evaluate(epoch)

            if args.checkpoints:
                torch.save(model.state_dict(), args.model_dir + f"/{epoch}.chkpt")

            logging.info('Epoch: {}/{},'.format(epoch, args.epochs))
            logging.info('train loss: {}, train acc: {},'.format(train_loss, train_acc))
            logging.info('test loss: {}, test acc: {}.'.format(test_loss, test_acc))

    return model


train_dataset, test_dataset = get_data()
model, loss, optimizer, device = get_model()
train_model(model, loss, optimizer, train_dataset, test_dataset, True, device)

logging.info('start saving')
# export model to gcs using GCSFuse
logging.info("Exporting model artifacts ...")
gs_prefix = 'gs://'
gcsfuse_prefix = '/gcs/'
if args.model_dir.startswith(gs_prefix):
    args.model_dir = args.model_dir.replace(gs_prefix, gcsfuse_prefix)
    dirpath = os.path.split(args.model_dir)[0]
    if not os.path.isdir(dirpath):
        os.makedirs(dirpath)

gcs_model_path = os.path.join(os.path.join(args.model_dir, 'model.pth'))
torch.save(model.state_dict(), gcs_model_path)
logging.info(f'Model is saved to {args.model_dir}')

Writing custom/trainer/task.py


### Test training package locally

Next, test your completed training package locally with just a few epochs.

In [17]:
! python3 custom/trainer/task.py --model-dir=custom --distribute=mirror --checkpoints=True

170499072it [00:03, 46602683.76it/s]                                            
Extracting ./train/cifar-10-python.tar.gz to ./train
INFO:root:Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./train
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )
170499072it [00:03, 55817665.92it/s]                                            
Extracting ./test/cifar-10-python.tar.gz to ./test
INFO:root:Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./test
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )
INFO:root:Get model architecture
INFO:root:Device: cpu
INFO:root:Epoch: 1, Training ...
INFO:root:Epoch: 1/20,
INFO:root:train loss: 2.312202, train acc: 6.25%,
INFO:root:test loss: 95946222338048.000000, test acc: 12.50%.
INFO:root:E

#### Store training script on your Cloud Storage bucket

Next, you package the training folder into a compressed tar ball, and then store it in your Cloud Storage bucket.

In [18]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_cifar10.tar.gz

custom/
custom/20.chkpt
custom/PKG-INFO
custom/README.md
custom/11.chkpt
custom/8.chkpt
custom/7.chkpt
custom/9.chkpt
custom/17.chkpt
custom/10.chkpt
custom/19.chkpt
custom/setup.py
custom/1.chkpt
custom/14.chkpt
custom/model.pth
custom/4.chkpt
custom/16.chkpt
custom/15.chkpt
custom/3.chkpt
custom/setup.cfg
custom/6.chkpt
custom/trainer/
custom/trainer/task.py
custom/trainer/__init__.py
custom/12.chkpt
custom/5.chkpt
custom/18.chkpt
custom/2.chkpt
custom/13.chkpt
Copying file://custom.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  4.2 MiB/  4.2 MiB]                                                
Operation completed over 1 objects/4.2 MiB.                                      


### Make Pytorch container for prediction

Currently, Vertex AI does not have a predefined container for making predictions with a deployed Pytorch model. No problem, you can assemble your own custom container. Typically, one would base the container on the `Torch Server`. For demonstration purpose, you build a placeholder container (not complete) that includes the latest `Torch Server` image, and push it to the `Container Registry`.

In [19]:
%%writefile Dockerfile

FROM pytorch/torchserve:latest-cpu


# run Torchserve HTTP serve to respond to prediction requests
CMD ["torchserve", \n     "--start", \n     "--ts-config=/home/model-server/config.properties", \n     "--models", \n     "$APP_NAME=$APP_NAME.mar", \n     "--model-store", \n     "/home/model-server/model-store"]

Writing Dockerfile


In [20]:
APP_NAME = "cifar10"
DEPLOY_IMAGE = f"gcr.io/{PROJECT_ID}/pytorch_predict_{APP_NAME}"
print(DEPLOY_IMAGE)

! docker build --tag=$DEPLOY_IMAGE ./

! docker push $DEPLOY_IMAGE

gcr.io/vertex-ai-dev/pytorch_predict_cifar10
Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?
Using default tag: latest
Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?


### Create and run custom training job


To train a custom model, you perform two steps: 1) create a custom training job, and 2) run the job.

#### Create custom training job

A custom training job is created with the `CustomTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the custom training job.
- `container_uri`: The training container image.

- `python_package_gcs_uri`: The location of the Python training package as a tarball.
- `python_module_name`: The relative path to the training script in the Python package.
- `model_serving_container_uri`: The container image for deploying the model.

*Note:* There is no requirements parameter. You specify any requirements in the `setup.py` script in your Python package.

In [31]:
DISPLAY_NAME = "cifar10_" + TIMESTAMP

job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=DISPLAY_NAME,
    python_package_gcs_uri=f"{BUCKET_URI}/trainer_cifar10.tar.gz",
    python_module_name="trainer.task",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
    project=PROJECT_ID,
)

### Prepare your command-line arguments

Now define the command-line arguments for your custom training container:

- `args`: The command-line arguments to pass to the executable that is set as the entry point into the container.
  - `--model-dir` : For our demonstrations, we use this command-line argument to specify where to store the model artifacts.
      - direct: You pass the Cloud Storage location as a command line argument to your training script (set variable `DIRECT = True`), or
      - indirect: The service passes the Cloud Storage location as the environment variable `AIP_MODEL_DIR` to your training script (set variable `DIRECT = False`). In this case, you tell the service the model artifact location in the job specification.
  - `--BLAH`:

In [22]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, TIMESTAMP)

DIRECT = False
if DIRECT:
    CMDARGS = ["--model_dir=" + MODEL_DIR]
else:
    CMDARGS = []

#### Run the custom training job

Next, you run the custom job to start the training job by invoking the method `run`, with the following parameters:

- `model_display_name`: The human readable name for the `Model` resource.
- `args`: The command-line arguments to pass to the training script.
- `replica_count`: The number of compute instances for training (replica_count = 1 is single node training).
- `machine_type`: The machine type for the compute instances.
- `accelerator_type`: The hardware accelerator type.
- `accelerator_count`: The number of accelerators to attach to a worker replica.
- `base_output_dir`: The Cloud Storage location to write the model artifacts to.
- `sync`: Whether to block until completion of the job.

In [23]:
if TRAIN_GPU:
    model = job.run(
        model_display_name="cifar10_" + TIMESTAMP,
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=False,
    )
else:
    model = job.run(
        model_display_name="cifar10_" + TIMESTAMP,
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=False,
    )

model_path_to_deploy = MODEL_DIR

INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://vertex-ai-devaip-20220324060506/20220324060506 
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/2483058445087932416?project=931647533046
INFO:google.cloud.aiplatform.training_jobs:CustomPythonPackageTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/2483058445087932416 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/6579504518646464512?project=931647533046
INFO:google.cloud.aiplatform.training_jobs:CustomPythonPackageTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/2483058445087932416 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:CustomPythonPackageTrainingJob projects/931647533046/locat

### List a custom training job

In [24]:
_job = job.list(filter=f"display_name={DISPLAY_NAME}")
print(_job)

resource name: projects/931647533046/locations/us-central1/trainingPipelines/2483058445087932416]


### Wait for completion of custom training job

Next, wait for the custom training job to complete. Alternatively, one can set the parameter `sync` to `True` in the `run()` method to block until the custom training job is completed.

In [25]:
model.wait()

### Delete a custom training job

After a training job is completed, you can delete the training job with the method `delete()`.  Prior to completion, a training job can be cancelled with the method `cancel()`.

In [26]:
job.delete()

INFO:google.cloud.aiplatform.base:Deleting CustomPythonPackageTrainingJob : projects/931647533046/locations/us-central1/trainingPipelines/2483058445087932416
INFO:google.cloud.aiplatform.base:Delete CustomPythonPackageTrainingJob  backing LRO: projects/931647533046/locations/us-central1/operations/90708128743555072
INFO:google.cloud.aiplatform.base:CustomPythonPackageTrainingJob deleted. . Resource name: projects/931647533046/locations/us-central1/trainingPipelines/2483058445087932416


# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Model
- Custom Job (Custom job deleted in previous cell)
- Cloud Storage Bucket

In [33]:
# Delete the model using the Vertex model object
model.delete()


if os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI

INFO:google.cloud.aiplatform.base:Deleting Model : projects/931647533046/locations/us-central1/models/7500485694641405952


NotFound: 404 The Model "projects/931647533046/locations/us-central1/models/7500485694641405952" does not exist.